###Connexion à la base de données PostgreSQL

In [1]:
import psycopg2

conn = psycopg2.connect(
        dbname="sentimentscore",
        user="postgres",
        password="rimthabet2000",
        host="localhost",
        port="5432"
    )
cursor = conn.cursor()

###Récupération de données JSON via une API avec requests

In [ ]:
from pip._vendor import requests

url = "http://192.168.1.74:8877/scrape-actualites"
reponse = requests.get(url)
actualites = reponse.json()
print(actualites)


ConnectTimeout: HTTPConnectionPool(host='192.168.1.74', port=8877): Max retries exceeded with url: /scrape-actualites (Caused by ConnectTimeoutError(<pip._vendor.urllib3.connection.HTTPConnection object at 0x00000170A60278C0>, 'Connection to 192.168.1.74 timed out. (connect timeout=None)'))

###Sauvegarder dans un fichier JSON

In [2]:
import json

with open('C:/Users/hp/Desktop/sentimentAnalaysis/actualites.json', 'r', encoding='utf-8') as file:
    actualites = json.load(file)

print("Données sauvegardées dans actualites.json")


# Sauvegarder dans un fichier
#with open("C:/Users/hp/Desktop/sentimentAnalaysis/actualites.json", "w", encoding="utf-8") as file:
   # json.dump(actualites, file, ensure_ascii=False, indent=4)

Données sauvegardées dans actualites.json


###Fonction de Conversion de Dates au Format Lisible

In [3]:
from datetime import datetime

def parse_date(date_str):
    try:
        # Remplacer 'T' par un espace et supprimer les millisecondes et le fuseau horaire
        parsed_date = datetime.fromisoformat(date_str.replace("Z", "+00:00"))
        # Convertir au format 'YYYY-MM-DD HH:MI:SS'
        return parsed_date.strftime('%Y-%m-%d %H:%M:%S')
    except (ValueError, TypeError):
        print(f"Date invalide : {date_str}")
        return None


###Extraction des Données Valides

In [6]:

resultats = []
for element in actualites:
    date_publication = element.get("datePublication")
    code_isin = element.get("isin")
    description = element.get("description")

    if not code_isin:
        print(f"ISIN manquant pour l'élément : {element}")

    date_publication = parse_date(date_publication)

    if date_publication and code_isin:
        resultats.append((date_publication, code_isin,description))


Date invalide : +127200-07-15T22:00:00.000+00:00
Date invalide : +178958987-07-05T22:00:00.000+00:00
Date invalide : +178958990-07-22T22:00:00.000+00:00
Date invalide : +178958990-07-22T22:00:00.000+00:00
Date invalide : +20162016-04-28T22:00:00.000+00:00
Date invalide : +1682183-12-03T23:00:00.000+00:00
Date invalide : +28109-08-26T22:00:00.000+00:00


###Insertion des Données Filtrées dans la Table actualite

In [7]:
insert_query = """
    INSERT INTO actualite (date_publication, code_isin,description)
    VALUES (%s, %s, %s)
"""

try:
    cursor.executemany(insert_query, resultats)
    conn.commit()
    print("Toutes les données valides ont été insérées avec succès.")
except Exception as e:
    print(f"Erreur lors de l'insertion : {e}")


Toutes les données valides ont été insérées avec succès.


###Fermeture de la Connexion à la Base de Données

In [ ]:
cursor.close()
conn.close()